#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import math
from collections import Counter
from datetime import datetime, timedelta
import matplotlib

# Lyrics Data

#### Import lyrics

In [2]:
lyrics = pd.read_csv('bruce_lyrics.csv')

#### Populate the album name and song name corresponding to each line of lyrics

In [3]:
for row,column in lyrics.iterrows():
    if isinstance(lyrics.loc[row,'Album'], basestring) is False:
        lyrics.loc[row,'Album'] = lyrics.loc[(row-1),'Album']
    else: pass
    
    if isinstance(lyrics.loc[row,'Song'], basestring) is False:
        lyrics.loc[row,'Song'] = lyrics.loc[(row-1),'Song']
    else: pass

#### Remove rows where the 'Lyrics' field is blank

In [4]:
for row, column in lyrics.iterrows():
    if isinstance(lyrics.loc[row,'Lyrics'], float):
        lyrics.drop(row, axis=0, inplace=True)

lyrics.reset_index(inplace=True)

#### Add a column with a unique ID for each song

In [5]:
lyrics['ID'] = lyrics['Song']
song_id = 1

In [6]:
for row, column in lyrics.iterrows():
    if row == 0:
        lyrics.loc[row,'ID'] = song_id
    else:
        if lyrics.loc[row,'Song'] == lyrics.loc[row-1,'Song']:
            lyrics.loc[row,'ID'] = song_id
        else:
            song_id += 1
            lyrics.loc[row,'ID'] = song_id

#### Create a list where each entry is a list of the lines in each song

In [7]:
all_lyrics = []; current_song = []; songID = 1
for row,column in lyrics.iterrows():
    if lyrics.loc[row,'ID'] == songID:
        current_song.append(lyrics.loc[row,'Lyrics'])
    else:
        all_lyrics.append(current_song)
        songID += 1
        current_song = [lyrics.loc[row,'Lyrics']]
all_lyrics.append(current_song)

#### Create a list where each entry is a list of the words in each song

In [8]:
all_lyrics_words = []
songs = range(0, len(all_lyrics))
for song in songs:
    all_lyrics_words.append(' '.join(all_lyrics[song]).split())

#### Create a new DataFrame ('lyrics_v2'), which contains just 1 row for each song

In [9]:
lyrics_v2 = lyrics.drop_duplicates(subset='ID')
lyrics_v2.reset_index(inplace=True)
lyrics_v2 = lyrics_v2.drop('level_0', axis=1).drop('index', axis=1).drop('Lyrics', axis=1)
lyrics_v2.head()

,Album,Song,ID
0,"Greetings from Asbury Park, N.J.",Blinded by the Light,1
1,"Greetings from Asbury Park, N.J.",Growin' Up,2
2,"Greetings from Asbury Park, N.J.",Mary Queen of Arkansas,3
3,"Greetings from Asbury Park, N.J.",Does this Bus Stop at 82nd Street?,4
4,"Greetings from Asbury Park, N.J.",Lost in the Flood,5


#### Add the lyrics to 'lyrics_v2'

In [10]:
lyrics_v2['Lyrics'] = pd.Series(all_lyrics_words)

In [11]:
lyrics_v2.head()

,Album,Song,ID,Lyrics
0,"Greetings from Asbury Park, N.J.",Blinded by the Light,1,"[Madman, drummers, bummers, and, Indians, in, ..."
1,"Greetings from Asbury Park, N.J.",Growin' Up,2,"[I, stood, stone-like, at, midnight, suspended..."
2,"Greetings from Asbury Park, N.J.",Mary Queen of Arkansas,3,"[Mary, Queen, of, Arkansas,, it's, not, too, e..."
3,"Greetings from Asbury Park, N.J.",Does this Bus Stop at 82nd Street?,4,"[Hey, bus, driver, keep, the, change,, bless, ..."
4,"Greetings from Asbury Park, N.J.",Lost in the Flood,5,"[The, ragamuffin, gunner, is, returnin', home,..."


#### Export data to .csv file to remove unwanted symbols at the end of each lyrics list using the "Replace" function in Excel. (These symbols are an artifact of the process used to collect lyrics from the internet.)

#### In addition, add the length to each song. These values were manually imported from tracklists on Wikipedia.

In [12]:
lyrics_v2.to_csv('lyrics_v2.csv')

In [13]:
lyrics_v3 = pd.read_csv('lyrics_v2.csv')

In [14]:
lyrics_v3 = lyrics_v3.drop('Unnamed: 0', axis=1)
lyrics_v3.head()

,ID,Album,Song,Minutes,Seconds,Length,Lyrics
0,1,"Greetings from Asbury Park, N.J.",Blinded by the Light,5,4,304,"['Madman', 'drummers', 'bummers', 'and', 'Indi..."
1,2,"Greetings from Asbury Park, N.J.",Growin' Up,3,5,185,"['I', 'stood', 'stone-like', 'at', 'midnight',..."
2,3,"Greetings from Asbury Park, N.J.",Mary Queen of Arkansas,5,21,321,"['Mary', 'Queen', 'of', 'Arkansas,', ""it's"", '..."
3,4,"Greetings from Asbury Park, N.J.",Does this Bus Stop at 82nd Street?,2,5,125,"['Hey', 'bus', 'driver', 'keep', 'the', 'chang..."
4,5,"Greetings from Asbury Park, N.J.",Lost in the Flood,5,13,313,"['The', 'ragamuffin', 'gunner', 'is', ""returni..."


#### Calculate the number of words per song

In [15]:
lyrics_list = map(lambda item: item.replace("'", "").replace('"','').replace(',,', ','), lyrics_v3.loc[:,'Lyrics'])

In [16]:
lyrics_words = []
counter = []; total_words = []
repeat_words = []; percent_repeat = []

once = 0

for i in range(0,len(lyrics_v3)):
    once = 0
    lyrics_words.append( lyrics_list[i].replace(',,' , ',').split(',') )
    total_words.append( len(lyrics_words[i]) )
    counter.append( Counter(lyrics_words[i]) )
    for j in counter[i].values():
        if j==1: once += 1
        else: pass
    repeat_words.append( float(total_words[i] - once) )
    percent_repeat.append( 100 * repeat_words[i] / total_words[i] )

#### Append total words/song, repeat words/song, and percentage of words that are repeated/song to the lyrics database

In [17]:
lyrics_v4 = pd.merge(lyrics_v3, pd.DataFrame({'repeat_words':repeat_words,'total_words':total_words,'percent_repeat':percent_repeat}), \
                left_index=True, right_index=True)
lyrics_v4.head()

,ID,Album,Song,Minutes,Seconds,Length,Lyrics,percent_repeat,repeat_words,total_words
0,1,"Greetings from Asbury Park, N.J.",Blinded by the Light,5,4,304,"['Madman', 'drummers', 'bummers', 'and', 'Indi...",57.782101,297.0,514
1,2,"Greetings from Asbury Park, N.J.",Growin' Up,3,5,185,"['I', 'stood', 'stone-like', 'at', 'midnight',...",54.440154,141.0,259
2,3,"Greetings from Asbury Park, N.J.",Mary Queen of Arkansas,5,21,321,"['Mary', 'Queen', 'of', 'Arkansas,', ""it's"", '...",57.383966,136.0,237
3,4,"Greetings from Asbury Park, N.J.",Does this Bus Stop at 82nd Street?,2,5,125,"['Hey', 'bus', 'driver', 'keep', 'the', 'chang...",28.901734,50.0,173
4,5,"Greetings from Asbury Park, N.J.",Lost in the Flood,5,13,313,"['The', 'ragamuffin', 'gunner', 'is', ""returni...",53.211009,232.0,436


# Album Data

#### Create a dataset of the number of songs per albums. Export this data to a .csv file, then manually add additional album attributes (date released and a field indicating if the album represented the release of new or previously creted material).

In [18]:
albums = lyrics_v3[['Album','ID']].groupby('Album').count()
albums.to_csv('albums.csv')
albums.head()

,ID
Album,
18 Tracks,18
Born in the U.S.A.,12
Born to Run,8
Darkness on the Edge of Town,10
Devils & Dust,12


In [20]:
albums = pd.read_csv('bruce_albums.csv')

In [21]:
albums.head()

,Album,Release Year,New Material,Release Date
0,"Greetings from Asbury Park, N.J.",1973,1,1/5/73
1,"The Wild, the Innocent, and the E Street Shuffle",1973,1,9/11/73
2,Born to Run,1975,1,8/25/75
3,Darkness on the Edge of Town,1978,1,6/2/78
4,The River,1980,1,10/17/80


#### We need to convert the elements in 'Release Date' to a time format readable by Pandas graphing functions.

In [22]:
date_released = np.array(albums.loc[:,'Release Date'])
formatted_dates = map(lambda date: datetime.strptime(date, '%m/%d/%y'), date_released)
formatted_dates = matplotlib.dates.date2num(formatted_dates)
type(formatted_dates)

numpy.ndarray

In [23]:
dates_series = pd.DataFrame({'date_number':formatted_dates})
albums = pd.merge(albums, dates_series, left_index=True, right_index=True)

In [24]:
albums.head()

,Album,Release Year,New Material,Release Date,date_number
0,"Greetings from Asbury Park, N.J.",1973,1,1/5/73,720263.0
1,"The Wild, the Innocent, and the E Street Shuffle",1973,1,9/11/73,720512.0
2,Born to Run,1975,1,8/25/75,721225.0
3,Darkness on the Edge of Town,1978,1,6/2/78,722237.0
4,The River,1980,1,10/17/80,723105.0


# Merge Lyrics Data and Album Data

#### Merge the lyrics dataset and the albums data set

In [25]:
master_springsteen_data = lyrics_v4.merge(albums, on='Album', how='left')
master_springsteen_data.head()

,ID,Album,Song,Minutes,Seconds,Length,Lyrics,percent_repeat,repeat_words,total_words,Release Year,New Material,Release Date,date_number
0,1,"Greetings from Asbury Park, N.J.",Blinded by the Light,5,4,304,"['Madman', 'drummers', 'bummers', 'and', 'Indi...",57.782101,297.0,514,1973,1,1/5/73,720263.0
1,2,"Greetings from Asbury Park, N.J.",Growin' Up,3,5,185,"['I', 'stood', 'stone-like', 'at', 'midnight',...",54.440154,141.0,259,1973,1,1/5/73,720263.0
2,3,"Greetings from Asbury Park, N.J.",Mary Queen of Arkansas,5,21,321,"['Mary', 'Queen', 'of', 'Arkansas,', ""it's"", '...",57.383966,136.0,237,1973,1,1/5/73,720263.0
3,4,"Greetings from Asbury Park, N.J.",Does this Bus Stop at 82nd Street?,2,5,125,"['Hey', 'bus', 'driver', 'keep', 'the', 'chang...",28.901734,50.0,173,1973,1,1/5/73,720263.0
4,5,"Greetings from Asbury Park, N.J.",Lost in the Flood,5,13,313,"['The', 'ragamuffin', 'gunner', 'is', ""returni...",53.211009,232.0,436,1973,1,1/5/73,720263.0


#### Export complete dataset to .csv

In [26]:
master_springsteen_data.to_csv('master_springsteen_data.csv')